# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. Using DeepLabCut for 3D markerless pose estimation during behavior across species.

pre-print: https://www.biorxiv.org/content/10.1101/476531v1

#### 1. Setup and parameters


In [1]:
import warnings
warnings.simplefilter('ignore')
import deeplabcut
import pandas as pd
import ruamel.yaml
import sys
import os
import tensorflow as tf
import numpy as np
import re

wd=r"E:\Users\Phil\DeepLabCut\dev"
vid_directory=r"E:\Users\Phil\Jerboa"
experimenter='Andy'
markerlist = ['COM','cal1','cal2']
dotsize = 5
corner2move2 = 512

#### 3. Create new project (1 per animal)

In [2]:
task='arena_chase'
vid_list=[ vid_directory+r'\0426_0221F_4-1.mp4',
         vid_directory+r'\0426_0221F_C1_1.mp4']
path_config_file=deeplabcut.create_new_project(task,experimenter,vid_list, working_directory=wd,copy_videos=False) 

Created "E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-12-06\videos"
Created "E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-12-06\labeled-data"
Created "E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-12-06\training-datasets"
Created "E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-12-06\dlc-models"
Creating the symbolic link of the video


CalledProcessError: Command 'mklink E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-12-06\videos\0426_0221F_4-1.mp4 E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4' returned non-zero exit status 1.

In [7]:
path_config_file=wd+r'\arena_chase-Andy-2019-10-21\config.yaml'

#### 4. Overwrite default bodyparts with XROMM marker list

In [8]:
%%capture
config = ruamel.yaml.load(open(path_config_file))
config['bodyparts']=markerlist
config['dotsize']=dotsize
config['corner2move2']=[corner2move2,corner2move2]
ruamel.yaml.round_trip_dump(config, sys.stdout)
with open(path_config_file, 'w') as fp:
    ruamel.yaml.round_trip_dump(config, fp)
    fp.close()

#### 5. Extract frames from vid_list for training set

In [9]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file, userfeedback=False) 

Config file read successfully.
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 57.17  seconds.
Extracting and downsampling... 6853  frames from the video.


6853it [00:14, 457.73it/s]


Kmeans clustering ... (this might take a while)
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 101.54  seconds.
Extracting and downsampling... 12173  frames from the video.


12173it [00:26, 457.54it/s]


Kmeans clustering ... (this might take a while)

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [10]:
deeplabcut.label_frames(path_config_file) 

Found new frames..
You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


#### 8. Create training set

In [11]:
deeplabcut.create_training_dataset(path_config_file)

E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\training-datasets\iteration-0\UnaugmentedDataSet_arena_chaseOct21  already exists!
E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\dlc-models\iteration-0\arena_chaseOct21-trainset95shuffle1  already exists!
E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\dlc-models\iteration-0\arena_chaseOct21-trainset95shuffle1//train  already exists!
E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\dlc-models\iteration-0\arena_chaseOct21-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


#### 9. Start training

In [12]:
deeplabcut.train_network(path_config_file, displayiters=50, maxiters=100000)

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['COM', 'cal1', 'cal2'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_arena_chaseOct21\\arena_chase_Andy95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\LabAdmin\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_arena_chaseOct21\\Documentation_data-arena_chase_95shuffle

Starting with standard pose-dataset loader.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\LabAdmin\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 100000
Display_iters overwritten as 50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'E:\\Users\\Phil\\DeepLabCut\\dev\\arena_chase-Andy-2019-10-21\\dlc-models\\iteration-0\\arena_chaseOct21-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_hube

iteration: 50 loss: 0.0923 lr: 0.005
iteration: 100 loss: 0.0253 lr: 0.005
iteration: 150 loss: 0.0205 lr: 0.005
iteration: 200 loss: 0.0204 lr: 0.005
iteration: 250 loss: 0.0181 lr: 0.005
iteration: 300 loss: 0.0150 lr: 0.005
iteration: 350 loss: 0.0158 lr: 0.005
iteration: 400 loss: 0.0131 lr: 0.005
iteration: 450 loss: 0.0139 lr: 0.005
iteration: 500 loss: 0.0129 lr: 0.005
iteration: 550 loss: 0.0141 lr: 0.005
iteration: 600 loss: 0.0109 lr: 0.005
iteration: 650 loss: 0.0120 lr: 0.005
iteration: 700 loss: 0.0122 lr: 0.005
iteration: 750 loss: 0.0109 lr: 0.005
iteration: 800 loss: 0.0088 lr: 0.005
iteration: 850 loss: 0.0101 lr: 0.005
iteration: 900 loss: 0.0095 lr: 0.005
iteration: 950 loss: 0.0093 lr: 0.005
iteration: 1000 loss: 0.0084 lr: 0.005
iteration: 1050 loss: 0.0073 lr: 0.005
iteration: 1100 loss: 0.0094 lr: 0.005
iteration: 1150 loss: 0.0082 lr: 0.005
iteration: 1200 loss: 0.0080 lr: 0.005
iteration: 1250 loss: 0.0066 lr: 0.005
iteration: 1300 loss: 0.0068 lr: 0.005
iterat

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


In [10]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

TypeError: extract_outlier_frames() missing 1 required positional argument: 'videos'

In [9]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['COM', 'cal1', 'cal2'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_arena_chaseOct21\\arena_chase_Andy95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\LabAdmin\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_arena_chaseOct21\\Documentation_data-arena_chase_95shuffle

Running  DeepCut_resnet50_arena_chaseOct21shuffle1_100000  with # of trainingiterations: 100000
INFO:tensorflow:Restoring parameters from E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\dlc-models\iteration-0\arena_chaseOct21-trainset95shuffle1\train\snapshot-100000
Analyzing data...


40it [00:03, 12.41it/s]


Done and results stored for snapshot:  snapshot-100000
Results for 100000  training iterations: 95 1 train error: 1.9 pixels. Test error: 359.31  pixels.
With pcutoff of 0.1  train error: 1.9 pixels. Test error: 189.28 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


In [11]:
def listAllDataVideos(directory,skip="", codec=".avi"):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            print(name)
            if name.endswith(codec) and not re.search(skip,root):
                vid_list.append(os.path.join(root,name))
            else:
                continue
    return vid_list
                
vids_to_analyze = listAllDataVideos(vid_directory,"",".mp4")

0426_0221F_4-1.mp4
0426_0221F_C1_1.mp4


In [15]:
deeplabcut.analyze_videos(path_config_file,vid_list, videotype='.mp4')

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['COM', 'cal1', 'cal2'],
 'batch_size': 8,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_arena_chaseOct21\\arena_chase_Andy95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\LabAdmin\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_arena_chaseOct21\\Documentation_data-arena_chase_95shuffle

Using snapshot-100000 for model E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\dlc-models\iteration-0\arena_chaseOct21-trainset95shuffle1
num_outputs =  1
INFO:tensorflow:Restoring parameters from E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-21\dlc-models\iteration-0\arena_chaseOct21-trainset95shuffle1\train\snapshot-100000
Starting to analyze %  E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4
Loading  E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4
Duration of video [s]:  57.17 , recorded with  119.88 fps!
Overall # of frames:  6853  found with (before cropping) frame dimensions:  1280 720
Starting to extract posture


6868it [04:40, 24.08it/s]                                                                                              

Detected frames:  6853


6868it [04:42, 24.32it/s]


...

12221it [08:17, 24.29it/s]                                                                                             

Detected frames:  12173


12221it [08:20, 24.40it/s]


Saving results in E:\Users\Phil\Jerboa...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DeepCut_resnet50_arena_chaseOct21shuffle1_100000'

In [16]:
deeplabcut.create_labeled_video(path_config_file,vid_list)

Starting %  E:\Users\Phil\Jerboa ['E:\\Users\\Phil\\Jerboa\\0426_0221F_4-1.mp4', 'E:\\Users\\Phil\\Jerboa\\0426_0221F_C1_1.mp4']
Loading  E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4 and data.
False 0 1280 0 720
6853
Duration of video [s]:  57.17 , recorded with  119.88 fps!
Overall # of frames:  6853 with cropped frame dimensions:  1280 720
Generating frames and creating video.


100%|█████████████████████████████████████████████████████████████████████████████| 6853/6853 [00:38<00:00, 176.22it/s]


Starting %  E:\Users\Phil\Jerboa ['E:\\Users\\Phil\\Jerboa\\0426_0221F_4-1.mp4', 'E:\\Users\\Phil\\Jerboa\\0426_0221F_C1_1.mp4']
Loading  E:\Users\Phil\Jerboa\0426_0221F_C1_1.mp4 and data.
False 0 1280 0 720
12173
Duration of video [s]:  101.54 , recorded with  119.88 fps!
Overall # of frames:  12173 with cropped frame dimensions:  1280 720
Generating frames and creating video.


100%|███████████████████████████████████████████████████████████████████████████| 12173/12173 [01:10<00:00, 173.75it/s]


## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [48]:
deeplabcut.extract_outlier_frames(path_config_file,[vids_to_analyze[0]])

network parameters: DeepCut_resnet50_arena_chaseOct10shuffle1_100000
Method  jump  found  2214  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


yes/no yes


Frames from video 0426_0221F_4-1  already extracted (more will be added)!
Loading video...
Duration of video [s]:  57.165441666666666 , recorded @  119.88011988011988 fps!
Overall # of frames:  6853 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 57.17  seconds.
Extracting and downsampling... 2214  frames from the video.


2214it [01:21, 27.17it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [5410, 1017, 5208, 1220, 6758, 306, 5078, 604, 3875, 3792, 2307, 5479, 5263, 5971, 1163, 3627, 821, 1348, 2369, 4597]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\0426_0221F_4-1.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [49]:
%gui wx
deeplabcut.refine_labels(path_config_file)

Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [50]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [51]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [52]:
deeplabcut.train_network(path_config_file, maxiters=100000, displayiters=50)

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['COM', 'wand_1', 'wand_2'],
 'batch_size': 8,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_arena_chaseOct10\\arena_chase_Andy95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\LabAdmin\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_arena_chaseOct10\\Documentation_data-arena_chase_95shu

Starting with standard pose-dataset loader.
INFO:tensorflow:Restoring parameters from C:\Users\LabAdmin\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 100000
Display_iters overwritten as 50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'E:\\Users\\Phil\\DeepLabCut\\dev\\arena_chase-Andy-2019-10-10\\dlc-models\\iteration-1\\arena_chaseOct10-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test'

iteration: 50 loss: 0.0881 lr: 0.005
iteration: 100 loss: 0.0283 lr: 0.005
iteration: 150 loss: 0.0235 lr: 0.005
iteration: 200 loss: 0.0188 lr: 0.005
iteration: 250 loss: 0.0195 lr: 0.005
iteration: 300 loss: 0.0193 lr: 0.005
iteration: 350 loss: 0.0166 lr: 0.005
iteration: 400 loss: 0.0172 lr: 0.005
iteration: 450 loss: 0.0156 lr: 0.005
iteration: 500 loss: 0.0174 lr: 0.005
iteration: 550 loss: 0.0158 lr: 0.005
iteration: 600 loss: 0.0136 lr: 0.005
iteration: 650 loss: 0.0146 lr: 0.005
iteration: 700 loss: 0.0126 lr: 0.005
iteration: 750 loss: 0.0128 lr: 0.005
iteration: 800 loss: 0.0115 lr: 0.005
iteration: 850 loss: 0.0128 lr: 0.005
iteration: 900 loss: 0.0109 lr: 0.005
iteration: 950 loss: 0.0111 lr: 0.005
iteration: 1000 loss: 0.0104 lr: 0.005
iteration: 1050 loss: 0.0095 lr: 0.005
iteration: 1100 loss: 0.0115 lr: 0.005
iteration: 1150 loss: 0.0097 lr: 0.005
iteration: 1200 loss: 0.0090 lr: 0.005
iteration: 1250 loss: 0.0092 lr: 0.005
iteration: 1300 loss: 0.0094 lr: 0.005
iterat

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


In [56]:
deeplabcut.analyze_videos(path_config_file,[vids_to_analyze[0]])

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['COM', 'wand_1', 'wand_2'],
 'batch_size': 8,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_arena_chaseOct10\\arena_chase_Andy95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\LabAdmin\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_arena_chaseOct10\\Documentation_data-arena_chase_95shu

Using snapshot-100000 for model E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-10\dlc-models\iteration-1\arena_chaseOct10-trainset95shuffle1
num_outputs =  1
INFO:tensorflow:Restoring parameters from E:\Users\Phil\DeepLabCut\dev\arena_chase-Andy-2019-10-10\dlc-models\iteration-1\arena_chaseOct10-trainset95shuffle1\train\snapshot-100000
Starting to analyze %  E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4
Loading  E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4
Duration of video [s]:  57.17 , recorded with  119.88 fps!
Overall # of frames:  6853  found with (before cropping) frame dimensions:  1280 720
Starting to extract posture


6868it [04:41, 23.87it/s]                                                                                              

Detected frames:  6853


6868it [04:43, 24.23it/s]


Saving results in E:\Users\Phil\Jerboa...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DeepCut_resnet50_arena_chaseOct10shuffle1_100000'

In [57]:
deeplabcut.create_labeled_video(path_config_file,[vids_to_analyze[0]])

Starting %  E:\Users\Phil\Jerboa ['E:\\Users\\Phil\\Jerboa\\0426_0221F_4-1.mp4']
Loading  E:\Users\Phil\Jerboa\0426_0221F_4-1.mp4 and data.
False 0 1280 0 720
6853
Duration of video [s]:  57.17 , recorded with  119.88 fps!
Overall # of frames:  6853 with cropped frame dimensions:  1280 720
Generating frames and creating video.


100%|█████████████████████████████████████████████████████████████████████████████| 6853/6853 [00:40<00:00, 168.66it/s]


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [38]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

NameError: name 'videofile_path' is not defined

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [7]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

UsageError: unrecognized arguments: #for making interactive plots.
